# Filtering Assembly based on blobplot results then doing Purge_dups


In [7]:
WKDIR=/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies
PURGEDUPS_BIN_DIR=/workspace/hraijc/git_clones/purge_dups/bin
PDDIR=${WKDIR}/purge_dups

FLYE10=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q10/Flye_BeeT_q10.fasta
BFLYE10=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q10/Flye_BeeT_q10_bfil.fasta

#FLYE15=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q15/Flye_BeeT_q15.fasta
HIFIASM18=/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.fasta
BHIFIASM18=/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR_bfil.fasta

ONTREADS_Q10=/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/fastq/BeeT_Q10.fastq
ONTREADS_Q15=/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/fastq/BeeT_Q15.fastq

ILLREAD1=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R1_001.clean.fastq
ILLREAD2=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R2_001.clean.fastq
ILLREADS=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61.clean.fastq

BLOBKEEP=/workspace/hraijc/Bee/BeeT/Assemblies/blobplots/Flye10.keep.txt
BLOBKEEP2=/workspace/hraijc/Bee/BeeT/Assemblies/blobplots/HiFi18.keep.txt

OUTPUT=/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT
cd ${WKDIR}

In [40]:
mkdir -p ${WKDIR}/log
mkdir -p ${PDDIR}
cd ${PDDIR}

## Remove contigs based on blobtools output

In [41]:
module load seqtk
#seqtk subseq ${FLYE10} ${BLOBKEEP} > ${BFLYE10}
seqtk subseq ${HIFIASM18} ${BLOBKEEP2} > ${BHIFIASM18}
#seqtk mergepe ${ILLREAD1} ${ILLREAD2} > ${ILLREADS}
module unload seqtk

In [42]:
du -sh ${HIFIASM18}
du -sh ${BHIFIASM18}
grep ">" ${HIFIASM18} | wc -l
grep ">" ${BHIFIASM18} | wc -l
#du -sh $ILLREADS

417M	/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.fasta
315M	/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR_bfil.fasta
4993
3368


## Purge_dups

Trialing a few different inputs for the coverage calculations on Purge_dups. Does it matter if its ONT or illumina data??

Cant be just illumina. must be combo of Illumina and long reads


 - Doing ONT_q15 Plus Illumina (10x coverage) (THIS DOES NOT SEEM TO WORK)
 - ONT_Q10 only (WORKED SLIGHLY BETTER)
 - ONT_Q15 only
 - HiFiasm18 with ont_Q10 data.


In [ ]:
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py --version
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py --help
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py --help


In [43]:
printf "${ONTREADS_Q10}" > ${PDDIR}/ONT10_files.txt
printf "${ONTREADS_Q15}" > ${PDDIR}/ONT15_files.txt
printf "${ILLREADS}" > ${PDDIR}/ill_files.txt

In [44]:
#python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.FLYE10.q10.json ${FLYE10} ${PDDIR}/ONT10_files.txt
#python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.FLYE10.q15.json ${FLYE10} ${PDDIR}/ONT15_files.txt
#python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.FLYE10.q15ill.json -s ${PDDIR}/ill_files.txt ${FLYE10} ${PDDIR}/ONT15_files.txt
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.BHIFIASM18.q10.json ${BHIFIASM18} ${PDDIR}/ONT10_files.txt


In [12]:
####Change json files####


In [45]:
cd ${PDDIR}


for FILE in *.json;
do 
    PRE=${FILE%.json}
    OUTPREFIX=${PRE#config.}
    echo python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py $FILE ${PURGEDUPS_BIN_DIR} ${OUTPREFIX} -p bash;
done 

python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.BHIFIASM18.q10.json /workspace/hraijc/git_clones/purge_dups/bin BHIFIASM18.q10 -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q10.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q10 -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q15ill.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q15ill -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q15.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q15 -p bash


In [46]:

sbatch << EOF
#!/bin/bash
#SBATCH -J PurgeDups
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=30G
#SBATCH --time=11:00:00

module load zlib
module load minimap2/2.22


#python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q15ill.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q15ill -p bash
#python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q10.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q10 -p bash
#python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.q15.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10.q15 -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.BHIFIASM18.q10.json /workspace/hraijc/git_clones/purge_dups/bin BHIFIASM18.q10 -p bash
EOF

Submitted batch job 1404999


In [8]:
## Organize results.
mkdir -p ${OUTPUT}/other
# Move raw assemblies
cp ${FLYE10} ${OUTPUT}/other/BeeT_FLYE10.fasta
cp ${HIFIASM18} ${OUTPUT}/other/BeeT_HIFIASM18.fasta
# Move manually filtered assemblies
cp ${BFLYE10} ${OUTPUT}/other/BeeT_FLYE10_bfil.fasta
cp ${BHIFIASM18} ${OUTPUT}/other/BeeT_HIFIASM18_bfil.fasta

# Move PURGED assemblies
cp ${PDDIR}/Flye10_BeeT_bfil_ONT10/seqs/*.purged.fa ${OUTPUT}/BeeT_FLYE10_bfil.purged.fasta
cp ${PDDIR}/BeeT_Hifiasm_q18l2kULR_bfil/seqs/*.purged.fa ${OUTPUT}/other/BeeT_HIFIASM18_bfil.purged.fasta



#Make a symlink of best assembly to make it easier to get stats for all.
ln -s ${OUTPUT}/BeeT_FLYE10_bfil.purged.fasta ${OUTPUT}/other/BeeT_FLYE10_bfil.purged.fasta

ln: failed to create symbolic link ‘/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT/other/BeeT_FLYE10.fasta’: File exists


: 1

In [15]:
echo "output/"
ls ${OUTPUT}
echo "output/other/"
ls ${OUTPUT}/other 

output/
BeeT_FLYE10_bfil.purged.fasta  other
output/other/
BeeT_FLYE10_bfil.fasta	       BeeT_HIFIASM18_bfil.fasta
BeeT_FLYE10_bfil.purged.fasta  BeeT_HIFIASM18_bfil.purged.fasta
BeeT_FLYE10.fasta	       BeeT_HIFIASM18.fasta


In [2]:
sbatch << EOF
#!/bin/bash
#SBATCH -J BUSCO
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem=10G
#SBATCH --time=21:00:00

module load BUSCO/v5.2.2 

cd /workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/


busco -m geno -c 12 -i ${BHIFIASM18} -l insecta_odb10 -o BeeT_Hifiasm_q18l2kULR_bfil.purged.busco
busco -m geno -c 12 -i ${BFLYE10} -l insecta_odb10 -o Flye10_BeeT_bfil_ONT10.purged.busco
busco -m geno -c 12 -i ${HIFIASM18} -l insecta_odb10 -o BeeT_Hifiasm_q18l2kULR.purged.busco



EOF

Submitted batch job 1428753


In [18]:
cd ${OUTPUT}/other 
module load BBMap/38.33

statswrapper.sh in=BeeT_FLYE10_bfil.fasta,BeeT_HIFIASM18_bfil.fasta,BeeT_FLYE10_bfil.purged.fasta,BeeT_HIFIASM18_bfil.purged.fasta,BeeT_FLYE10.fasta,BeeT_HIFIASM18.fasta format=5 
module unload BBMap/38.33


java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=BeeT_FLYE10_bfil.fasta,BeeT_HIFIASM18_bfil.fasta,BeeT_FLYE10_bfil.purged.fasta,BeeT_HIFIASM18_bfil.purged.fasta,BeeT_FLYE10.fasta,BeeT_HIFIASM18.fasta format=5
n_contigs	contig_bp	gap_pct	ctg_N50	ctg_L50	ctg_N90	ctg_L90	ctg_max	gc_avg	gc_std	filename
821	679462663	0.000	76	2766627	279	574703	11075509	0.34787	0.03873	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT/other/BeeT_FLYE10_bfil.fasta
3368	288138445	0.000	939	101219	2557	48153	545116	0.34288	0.02822	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT/other/BeeT_HIFIASM18_bfil.fasta
652	673910815	0.000	75	2770209	270	589332	11075509	0.34780	0.03128	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT/BeeT_FLYE10_bfil.purged.fasta
3165	275812870	0.000	882	103454	2408	48353	545116	0.34311	0.02161	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/OUTPUT/other/BeeT_HIFIASM18_bfil.purged.fasta
3262	696244996	0.000	79	

| n_contigs 	| contig_bp 	| ctg_N50 	| ctg_L50 	| ctg_N90 	| ctg_L90 	| ctg_max  	| gc_avg  	| filename                         	| Busco                                         	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
|-----------	|-----------	|---------	|---------	|---------	|---------	|----------	|---------	|----------------------------------	|-----------------------------------------------	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 3262      	| 696244996 	| 79      	| 2696791 	| 308     	| 449130  	| 11075509 	| 0.35009 	| BeeT_FLYE10.fasta                	|                                               	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
| 821       	| 679462663 	| 76      	| 2766627 	| 279     	| 574703  	| 11075509 	| 0.34787 	| BeeT_FLYE10_bfil.fasta           	|                                               	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
| 652       	| 673910815 	| 75      	| 2770209 	| 270     	| 589332  	| 11075509 	| 0.3478  	| BeeT_FLYE10_bfil.purged.fasta    	| C:98.9%[S:98.6%,D:0.3%],F:0.2%,M:0.9%,n:1367  	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
| 4993      	| 380881427 	| 1364    	| 92058   	| 3749    	| 42794   	| 545116   	| 0.3438  	| BeeT_HIFIASM18.fasta             	| C:69.5%[S:66.0%,D:3.5%],F:1.5%,M:29.0%,n:1367 	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
| 3368      	| 288138445 	| 939     	| 101219  	| 2557    	| 48153   	| 545116   	| 0.34288 	| BeeT_HIFIASM18_bfil.fasta        	|                                               	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
| 3165      	| 275812870 	| 882     	| 103454  	| 2408    	| 48353   	| 545116   	| 0.34311 	| BeeT_HIFIASM18_bfil.purged.fasta 	| C:66.8%[S:65.8%,D:1.0%],F:1.5%,M:31.7%,n:1367 	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|
|           	|           	|         	|         	|         	|         	|          	|         	|                                  	|                                               	|   	|   	|   	|   	|   	|   	|   	|   	|   	|   	|